In [17]:
import csv
import copy
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
market_csv_file = "data/S&P500.csv"
stock_csv_file = "data/stock_clean.csv"
df_market = pd.read_csv(market_csv_file)
df_stock = pd.read_csv(stock_csv_file)

In [19]:
df_stock = df_stock.rename(columns = {df_stock.columns[0]:"date"})
result = df_stock["date"].isin(["19720731"])
t = df_stock["date"][result].index.values[0]
print(t)

119


In [20]:
#Drop columns that have less than pre 120 + post 12 = 132 valid return
tmp_df_stock_all = df_stock[t-119:t+13].dropna(thresh=df_stock[t-119:t+13].shape[0],how='all',axis=1)
#print(tmp_df)
#Get the prev
tmp_df_stock = tmp_df_stock_all[0:120]
tmp_df_market = df_market[t-119:t+1]
N = len(tmp_df_stock.columns[1:])

In [35]:
print(tmp_df_stock[tmp_df_stock.columns[1:2]].values)
print(tmp_df_market[tmp_df_market.columns[1:]].values)

[[ 0.003704]
 [-0.054004]
 [ 0.070866]
 [ 0.080882]
 [ 0.058319]
 [ 0.047002]
 [ 0.0387  ]
 [-0.009009]
 [ 0.084848]
 [ 0.09162 ]
 [-0.001289]
 [-0.045161]
 [ 0.121081]
 [-0.07767 ]
 [ 0.057895]
 [ 0.246766]
 [-0.004016]
 [ 0.068548]
 [ 0.043774]
 [ 0.      ]
 [ 0.003636]
 [-0.01087 ]
 [ 0.075646]
 [ 0.073756]
 [ 0.01278 ]
 [ 0.074722]
 [ 0.029586]
 [-0.023276]
 [-0.05638 ]
 [ 0.053459]
 [-0.02597 ]
 [ 0.040248]
 [ 0.084821]
 [-0.043621]
 [-0.104348]
 [ 0.009709]
 [ 0.059615]
 [-0.012232]
 [ 0.108359]
 [-0.009497]
 [ 0.076923]
 [ 0.089947]
 [-0.020388]
 [-0.0475  ]
 [ 0.002625]
 [ 0.006283]
 [ 0.057895]
 [-0.014925]
 [-0.135354]
 [-0.044379]
 [-0.024768]
 [ 0.013968]
 [-0.028571]
 [ 0.140523]
 [-0.036103]
 [ 0.156627]
 [ 0.052083]
 [-0.018812]
 [ 0.061224]
 [ 0.040865]
 [-0.066051]
 [-0.0125  ]
 [-0.040506]
 [-0.043799]
 [ 0.002793]
 [-0.041783]
 [-0.036628]
 [-0.024465]
 [ 0.203762]
 [-0.017188]
 [ 0.292225]
 [ 0.039419]
 [-0.100998]
 [-0.058296]
 [ 0.047619]
 [ 0.101818]
 [ 0.04375 ]

In [64]:
y = tmp_df_stock[tmp_df_stock.columns[1:]].values
y_bar = y.mean()
x = tmp_df_market[tmp_df_market.columns[1:]].values
x_bar = x.mean()
beta_1 = np.sum((y-y_bar)*(x),axis=0)/np.sum((x-x_bar)*x,axis=0)
beta_0 = y_bar - beta_1*x_bar
residual = y - (beta_1*x+beta_0)
delta = np.sum(residual**2,axis=0)/(N-2)
D = np.diag(delta)
s_2 = x.var()
F = s_2*beta_1@beta_1.T + D
#print(F)
#stats.linregress(np.tile(tmp_df_market[tmp_df_market.columns[1]].values,N).reshape(120,N),tmp_df_stock[tmp_df_stock.columns[1:]].values)

In [68]:
cov_matrix = F
cov_matrix_inv = np.linalg.pinv(cov_matrix)#pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
w = cov_matrix_inv@np.ones((N,1))/(np.ones((1,N))@cov_matrix_inv@np.ones((N,1)))
mu = tmp_df_stock[tmp_df_stock.columns[1:]].mean()
#annual_return = (tmp_df_.astype(np.float64)+1).prod(axis=0)-1
#annual_returns.append(annual_return@w)
#print(annual_return[0:10])
print(np.sum(w))
print(np.min(w))
print(mu@w)

1.0000000000101852
6.15674633638902e-05
[0.00946186]


In [105]:
annual_returns = []
for i in tqdm(range(t,384,12)):
    #tmp_df = df[i-119:i+1].dropna(thresh=df[i-119:i+1].shape[0],how='all',axis=1)
    #Drop columns that have less than pre 120 + post 12 = 132 valid return
    tmp_df_stock_all = df_stock[i-119:i+13].dropna(thresh=df_stock[i-119:i+13].shape[0],how='all',axis=1)
    #print(tmp_df)
    #Get the prev
    tmp_df_stock = tmp_df_stock_all[0:120]
    N = len(tmp_df_stock.columns[1:])
    tmp_df_market = df_market[t-119:t+1]
    
    y = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).values
    y_bar = y.mean()
    x = tmp_df_market[tmp_df_market.columns[1:]].astype(np.float64).values
    x_bar = x.mean()
    beta_1 = np.sum((y-y_bar)*(x),axis=0)/np.sum((x-x_bar)*x,axis=0)
    beta_0 = y_bar - beta_1*x_bar
    residual = y - (beta_1*x+beta_0)
    delta = np.sum(residual**2,axis=0)/(N-2)
    D = np.diag(delta)
    s_2 = x.var()
    cov_matrix = s_2*beta_1[:,np.newaxis]@beta_1[:,np.newaxis].T + D
    print((beta_1[:,np.newaxis]@beta_1[:,np.newaxis].T).shape)
    #print(D)
    cov_matrix_inv = np.linalg.pinv(cov_matrix)#pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    w = cov_matrix_inv@np.ones((N,1))/(np.ones((1,N))@cov_matrix_inv@np.ones((N,1)))
    mu = tmp_df_stock[tmp_df_stock.columns[1:]].mean()
    for j in range(12):
        tmp_df_post = tmp_df_stock_all[120+j:121+j]
        tmp_df_post = tmp_df_post[tmp_df_post.columns[1:]]
        #print(tmp_df_post)
        annual_return = (tmp_df_post.astype(np.float64)+1).prod(axis=0)-1
        annual_returns.append(annual_return@w)
    #print(annual_return[0:10])
#     print(np.sum(w))
#     print(np.min(w))
#     print(mu@w)

  4%|▍         | 1/23 [00:00<00:04,  4.82it/s]

(810, 810)
(996, 996)


  9%|▊         | 2/23 [00:00<00:04,  4.34it/s]

(1049, 1049)


 13%|█▎        | 3/23 [00:00<00:05,  3.69it/s]

...

 91%|█████████▏| 21/23 [00:07<00:00,  3.28it/s]

(1040, 1040)


 96%|█████████▌| 22/23 [00:08<00:00,  3.32it/s]

(1040, 1040)


100%|██████████| 23/23 [00:08<00:00,  2.73it/s]


In [100]:
annual_returns = np.array(annual_returns)
annual_returns.std()*np.sqrt(12)

0.143150006052536

In [101]:
annual_returns.mean()

0.013253112485461677